In [1]:
# %% General Modules
import numpy as np
import pandas as pd
import os
import random
# %% Special Modules - deep learning
import torch
from torch.utils.data import TensorDataset



# %% OnI: Instance Level Data
# def filter_bag_inst(path, filt):
#     data = pd.read_csv(path)
#     #print(dataInst.shape)
    
#     if filt:
#         dict1 = data.groupby('bag')['bag'].agg('count').to_dict()
#         l = [k for k, v in dict1.items() if v < 9]
#         data1 = data[~data['bag'].isin(l)]
#     else:
#         data1 = data
        
#     return data1
# def seed_everything(seed: int = 42):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)

In [2]:
# dict1 = dataInst.groupby('bag')['bag'].agg('count').to_dict()

# [k for k, v in dict1.items() if v < 4]

In [3]:
dataInst = pd.read_csv("pca_inst.csv")

In [4]:
dataInst['case'] = dataInst['bag']

In [5]:
# dataInst = filter_bag_inst("inst.csv", True)

In [6]:
dataInst.head()

,label,bag,inst,img,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,...,principal component 42,principal component 43,principal component 44,principal component 45,principal component 46,principal component 47,principal component 48,principal component 49,principal component 50,case
0,CTR,1,1,10_S_3_0.jpeg,-0.886563,-9.814194,4.119864,1.056631,0.618267,-0.480605,...,0.314288,-0.272947,0.865218,-0.408135,-0.692384,0.403613,0.067921,0.342031,0.492849,1
1,CTR,1,2,10_S_3_1.jpeg,19.623776,-0.405351,-6.345954,3.078536,1.737264,-0.872794,...,0.995767,-1.235957,-1.290956,0.205526,-0.270159,0.092696,-0.108957,1.211166,0.687318,1
2,CTR,1,3,10_S_3_2.jpeg,-8.707897,-1.831399,-2.654622,0.229464,-0.870441,3.716145,...,1.825389,-0.351197,-0.356001,-0.980885,-0.053434,-0.927345,0.168369,0.284159,1.225107,1
3,CTR,1,4,10_S_3_3.jpeg,0.145934,-1.110487,-4.999176,-0.542506,1.684357,-1.293804,...,-0.275685,0.835450,0.856794,-0.429234,-0.536420,-0.408148,-0.492305,0.626028,-0.097138,1
4,CTR,1,5,10_S_3_0.jpeg,-17.714116,-3.153717,-1.817899,-1.337777,-0.752238,4.027222,...,-0.430994,-0.034895,1.237139,-0.274066,-0.490047,0.082345,1.897989,-1.032462,0.285864,1


In [7]:
# %% Assign into NumPy Array
insts = np.array( dataInst.iloc[:,2] )
bags = np.array( dataInst.iloc[:,1] )
cases = np.array( dataInst[['case']] )
labels = np.array( dataInst.iloc[:,0] )
feats = np.array( dataInst.iloc[:,4:54])

# %% PyTorch Tensor
label2bool = np.array([True if l == 'CASE' else False for l in labels])
data = TensorDataset(torch.Tensor(feats), torch.Tensor(label2bool))

In [8]:
dataInst[['bag', 'case', 'label']].head()

,bag,case,label
0,1,1,CTR
1,1,1,CTR
2,1,1,CTR
3,1,1,CTR
4,1,1,CTR


In [9]:
# %% Bag Level Data
dataBag = dataInst[['bag', 'case', 'label']]
print( dataBag.shape )
###
dataBag = dataBag.drop_duplicates()
dataBag = dataBag.rename( columns={
	'bag': 'slide_id',
	'case': 'case_id'
} )
dataBag.to_csv("bag.csv", index = False)

(936, 3)


In [10]:
# %% OnI: Bag Level Data
# path = "{}/bag.csv".format( args.dir_oniput )
# print(path)
# dataBag.to_csv( path, index=False )

# %% List of Bags
bags_unique = np.array(dataBag['slide_id'])
print(bags_unique)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56]


In [11]:
# %% Instances in Every Bag
from utils.file_utils import load_pkl, save_pkl
dir = "/Users/chitrakbanerjee/Desktop/omics/glom_pca/bags"
print(dir)
os.makedirs( dir, exist_ok=True )
###
insts_in_bags = {}
for b in bags_unique: # this iterates 56 times
    # array of 1:56
    insts_sub = np.where(bags == b)[0]
    # for each of 56 subjects tensor of features
    feats_sub = data.tensors[0][insts_sub,:]
    # insts in bag
    insts_in_bags[b] = insts[insts_sub]
    path = "{}/{}.pt".format( dir, b )
    #print(path)
    torch.save( feats_sub, path )
    #print(insts_sub, feats_sub, insts_in_bags[b])
    #print(insts_sub.shape, feats_sub.shape, insts_in_bags[b].shape)

###### this code is redundant    
# %% Cell Counts
# for ( key, values ) in insts_in_bags.items():
# 	print( str(key) + ':' + str( len(values) ) )

# # %%
path = "/Users/chitrakbanerjee/Desktop/omics/glom_pca/insts_in_bags.pkl"
# print(path)
save_pkl( path, insts_in_bags )

# # %%

# # %% [markdown]
# ## EoF

/Users/chitrakbanerjee/Desktop/omics/glom_pca/bags


In [12]:
args = pd.DataFrame({'dir_project': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'dir_code': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'results_dir': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/result/',
        'dir_oniput': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'dir_split' : '/Users/chitrakbanerjee/Desktop/omics/glom_pca/split/',
        'dir_result' : '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'cell_barcode_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/insts_in_bags.pkl',
        'ckpt_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/0_1/s_0_checkpoint.pt',
        'feature_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/bags',
        'meta_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/bag.csv',
        'cv_fold': 3,
        #'cv_val': 0.3,
        'cv_test': 0.25,
        'cv_train': 1,
        'cv_equal': True,
        'opt': 'adam',
        'log_data': True,
        'n_classes': 2,
        'lr': 1e-4,
        'reg': 1e-5, # []
        'drop_out': True,
        'weighted_sample': True,
        'max_epochs': 400,
        'early_stopping': False,
        'model_type': 'clam_mb',
        'bag_loss': 'svm',
        'inst_loss': 'ce',
        'bag_weight': 0.7,
        'B': 2,
        'model_size': 'tiny',
        'no_inst_cluster': False,
        'subtyping': False,
        'log_data': True,
        'testing': False
       }, index = [0])


In [13]:
import os
os.makedirs( args['dir_split'].item(), exist_ok=True )
print(args['dir_split'].item())

# %% General Modules
import numpy as np
import pandas as pd
os.path.join('/Users/chitrakbanerjee/Desktop/omics/glom_pca/')
# %% Internal Modules
from datasets.dataset_generic import Generic_WSI_Classification_Dataset
from datasets.dataset_generic import save_splits

# %%

# %% [markdown]
## Preprocessing

# %% Data
# if ( args.job_data == '20210903-1-AT - AD' ):
n_classes = 2
seed =1
cv_fold =3
#cv_val =0.3
cv_test=0.25
cv_train =1
cv_equal = True

dataset = Generic_WSI_Classification_Dataset(csv_path = "bag.csv",
                        label_dict = { "CTR": 0, "CASE": 1},seed = 1, 
                                             patient_strat=True, ignore=[])
# elif ( args.job_data == '20230412-1-HM' ):
# 	args.n_classes = 2
# 	dataset = Generic_WSI_Classification_Dataset(
# 		csv_path = os.path.join( args.dir_oniput, 'bag.csv' ),
# 		label_dict = { 'CTR': 0, args.job_label: 1 },
# 		seed=args.seed, patient_strat=True, ignore=[]
# 	)
# else:
# 	raise NotImplementedError

# %%

# %% [markdown]
## Main


# %% Preparation
num_slides_cls = np.array([len(cls_ids) for cls_ids in dataset.patient_cls_ids])
#print(dataset.patient_cls_ids)
#val_num = np.round( num_slides_cls * cv_val ).astype(int)
test_num = np.round( num_slides_cls * cv_test ).astype(int)
###
if cv_equal is True:
#    val_num = [ min(val_num) for n in val_num ]
    test_num = [ min(test_num) for n in test_num ]
###
dataset.create_splits(
    k = cv_fold,
    test_num = test_num,
    label_frac = cv_train)
# dataset.create_splits_new(
#     k = cv_fold,
#     cv_test = cv_test)


# %% Run: Split
for f in range(cv_fold):
    dataset.set_splits()
    descriptor_df = dataset.test_split_gen( return_descriptor=True )
    splits = dataset.return_splits( from_id=True )
    filename = '{}_{}_{}'.format(
        int( 100 * cv_test ), cv_fold, f
    )
    save_splits( splits, ['train', 'test'], os.path.join(
        'split', filename + '.csv') )
    save_splits( splits, ['train', 'test'], os.path.join(
        'split', filename + '_bool' + '.csv'
    ), boolean_style=True )
    descriptor_df.to_csv( os.path.join(
        'split', filename + '_descriptor' + '.csv'
    ) )

# %%

# %% [markdown]
## EoF


/Users/chitrakbanerjee/Desktop/omics/glom_pca/split/
label column: label
label dictionary: {'CTR': 0, 'CASE': 1}
number of classes: 2
slide-level counts:  
 0    40
1    16
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 40
Slide-LVL; Number of samples registered in class 0: 40
Patient-LVL; Number of samples registered in class 1: 16
Slide-LVL; Number of samples registered in class 1: 16

number of training samples: 48
number of samples in cls 0: 36
number of samples in cls 1: 12

number of test samples: 8
number of samples in cls 0: 4
number of samples in cls 1: 4



number of training samples: 48
number of samples in cls 0: 36
number of samples in cls 1: 12

number of test samples: 8
number of samples in cls 0: 4
number of samples in cls 1: 4



number of training samples: 48
number of samples in cls 0: 36
number of samples in cls 1: 12

number of test samples: 8
number of samples in cls 0: 4
number of samples in cls 1: 4




In [14]:
# %% General Modules
import itertools
import math
import pdb
import random
###
import numpy as np
import pandas as pd

# %% Special Modules - deep learning
import torch
#from torch.utils.data import DataLoader, sampler
#import torch.nn as nn
#import torch.nn.functional as F
###
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

# %% Internal Modules
from datasets.dataset_generic import Generic_MIL_Dataset
from utils.core_utils import train
from utils.file_utils import load_pkl, save_pkl
from utils.utils import *

# %%

# %% [markdown]
## Preprocessing

# %% Data

dataset = Generic_MIL_Dataset(
		data_dir = '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
		csv_path = os.path.join( '/Users/chitrakbanerjee/Desktop/omics/glom_pca/', 'bag.csv' ),
		label_dict = { 'CTR': 0, "CASE": 1 },
		seed=1, patient_strat=True, ignore=[]
	)

# %%

# %% [markdown]
## Functions

# %% Seeding
def seed_torch(seed):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)
	if device.type == 'cuda':
		torch.cuda.manual_seed(seed)
		torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True

# %% Single Run of CLAM-SMILE
def runSingle( f, args ):
	( train_dataset, test_dataset ) = dataset.return_splits(
		from_id = False, 
		csv_path = os.path.join( '/Users/chitrakbanerjee/Desktop/omics/glom_pca/split/', '{}_{}_{}'.format(
			 int( 100 * cv_test ), cv_fold, f
		) + '.csv' ),
		 test_frac=args['cv_test'].item()
	)
	datasets = ( train_dataset, test_dataset )
	return train( datasets, f, args )

# %%

# %% [markdown]
## Main

# %% Main
if __name__ == "__main__":
    args.results_dir = '{}/{}_{}'.format( args['dir_result'].item(),
         int( 100 * args['cv_test'].item() ),
                                            args['cv_fold'].item())
    os.makedirs( args['results_dir'].item(), exist_ok=True )
    ###
    folds = np.arange( 0, args['cv_fold'].item())

    all_stopping_epoch = []
    all_train_auc = []
    all_train_acc = []
    all_test_auc = []
    all_test_acc = []
    ###
    all_train_acc_vec = []
    all_train_auc_vec = []
    ###
    for f in folds:
        (results, train_auc, train_acc, test_auc, test_acc,
         stopping_epoch, train_acc_vec, train_auc_vec) = runSingle( f, args )
        all_stopping_epoch.append(stopping_epoch)
        all_train_auc.append(train_auc)
        all_train_acc.append(train_acc)
        if cv_test > 0:
            all_test_auc.append(test_auc)
            all_test_acc.append(test_acc)
        else:
            all_test_auc.append(-1)
            all_test_acc.append(-1)
        all_train_acc_vec.append( train_acc_vec )
        all_train_auc_vec.append( train_auc_vec )
        ###
        results_dir = args['results_dir'].item()
        path = '{}/{}.pkl'.format( results_dir, f )
        save_pkl( path, results )
        ###
        conv = pd.DataFrame( {'train_acc': train_acc_vec, 'train_auc': train_auc_vec} )
        path = '{}/{}.csv'.format( results_dir, f )
        conv.to_csv(path)
    ###
    final_df = pd.DataFrame({'folds': folds, 'stopping_epoch': all_stopping_epoch,
        'train_auc': all_train_auc, 'train_acc' : all_train_acc,
                             'test_auc': all_test_auc, 'test_acc' : all_test_acc} )
    ###
    final_df = final_df.append( { 'folds': '', 'stopping_epoch': np.mean(all_stopping_epoch),
        'train_auc': np.mean(all_train_auc),
        'train_acc' : np.mean(all_train_acc),
        'test_auc': np.mean(all_test_auc),
        'test_acc': np.mean(all_test_acc)
    }, ignore_index = True )
    ###
    print(final_df)
    path = '{}/summary.csv'.format( results_dir )
    final_df.to_csv(path)
    ###
    print("finished!")
print("end script")

# %%

# %% [markdown]
## EoF


label column: label
label dictionary: {'CTR': 0, 'CASE': 1}
number of classes: 2
slide-level counts:  
 0    40
1    16
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 40
Slide-LVL; Number of samples registered in class 0: 40
Patient-LVL; Number of samples registered in class 1: 16
Slide-LVL; Number of samples registered in class 1: 16

Training Fold 0!

Init train/val/test splits... 
Done!
Training on 48 samples
Testing on 8 samples

Init loss function... Setting tau to 1.0
Done!

Init Model... Done!
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=50, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Sig

epoch:10


batch 19, loss: 0.4913, instance_loss: 0.1399, weighted_loss: 0.3859, label: 1, bag_size: 22
batch 39, loss: 1.0081, instance_loss: 4.9255, weighted_loss: 2.1833, label: 1, bag_size: 34


class 0 clustering acc 0.6770833333333334: correct 65/96
class 1 clustering acc 0.7083333333333334: correct 68/96
Epoch: 10, train_loss: 1.1854, train_clustering_loss:  0.9752, train_error: 0.3333
class 0: acc 0.5, correct 11/22
class 1: acc 0.8076923076923077, correct 21/26

Test Set, test_loss: 2.7552, test_error: 0.6250, auc: 0.3125
class 0 clustering acc 0.625: correct 10/16
class 1 clustering acc 0.8125: correct 13/16
class 0: acc 0.25, correct 1/4
class 1: acc 0.5, correct 2/4
epoch:11


batch 19, loss: 0.1003, instance_loss: 0.3562, weighted_loss: 0.1771, label: 0, bag_size: 7
batch 39, loss: 1.7421, instance_loss: 1.0330, weighted_loss: 1.5294, label: 0, bag_size: 2


class 0 clustering acc 0.6354166666666666: correct 61/96
class 1 clustering acc 0.75: correct 72/96
Epoch: 11, train

epoch:22


batch 19, loss: 0.4477, instance_loss: 0.2983, weighted_loss: 0.4029, label: 1, bag_size: 14
batch 39, loss: 0.1058, instance_loss: 0.3739, weighted_loss: 0.1862, label: 0, bag_size: 28


class 0 clustering acc 0.7395833333333334: correct 71/96
class 1 clustering acc 0.8229166666666666: correct 79/96
Epoch: 22, train_loss: 0.8466, train_clustering_loss:  0.8192, train_error: 0.2500
class 0: acc 0.5555555555555556, correct 10/18
class 1: acc 0.8666666666666667, correct 26/30

Test Set, test_loss: 2.5083, test_error: 0.6250, auc: 0.3125
class 0 clustering acc 0.625: correct 10/16
class 1 clustering acc 0.8125: correct 13/16
class 0: acc 0.25, correct 1/4
class 1: acc 0.5, correct 2/4
epoch:23


batch 19, loss: 0.1058, instance_loss: 1.0089, weighted_loss: 0.3767, label: 1, bag_size: 29
batch 39, loss: 0.1370, instance_loss: 0.0778, weighted_loss: 0.1192, label: 1, bag_size: 15


class 0 clustering acc 0.6770833333333334: correct 65/96
class 1 clustering acc 0.8229166666666666:

batch 19, loss: 0.0619, instance_loss: 0.0944, weighted_loss: 0.0716, label: 1, bag_size: 15
batch 39, loss: 0.0453, instance_loss: 1.3909, weighted_loss: 0.4490, label: 0, bag_size: 7


class 0 clustering acc 0.6770833333333334: correct 65/96
class 1 clustering acc 0.8020833333333334: correct 77/96
Epoch: 34, train_loss: 1.0105, train_clustering_loss:  0.8241, train_error: 0.2917
class 0: acc 0.6, correct 12/20
class 1: acc 0.7857142857142857, correct 22/28

Test Set, test_loss: 2.3480, test_error: 0.6250, auc: 0.3125
class 0 clustering acc 0.625: correct 10/16
class 1 clustering acc 0.8125: correct 13/16
class 0: acc 0.25, correct 1/4
class 1: acc 0.5, correct 2/4
epoch:35


batch 19, loss: 0.2004, instance_loss: 0.3204, weighted_loss: 0.2364, label: 0, bag_size: 24
batch 39, loss: 0.8058, instance_loss: 0.0953, weighted_loss: 0.5926, label: 1, bag_size: 20


class 0 clustering acc 0.6666666666666666: correct 64/96
class 1 clustering acc 0.7708333333333334: correct 74/96
Epoch: 35, t

epoch:46


batch 19, loss: 0.4891, instance_loss: 0.8366, weighted_loss: 0.5934, label: 0, bag_size: 20
batch 39, loss: 0.2024, instance_loss: 1.7054, weighted_loss: 0.6533, label: 1, bag_size: 29


class 0 clustering acc 0.6875: correct 66/96
class 1 clustering acc 0.6979166666666666: correct 67/96
Epoch: 46, train_loss: 0.7603, train_clustering_loss:  1.0828, train_error: 0.1875
class 0: acc 0.8076923076923077, correct 21/26
class 1: acc 0.8181818181818182, correct 18/22

Test Set, test_loss: 2.2132, test_error: 0.5000, auc: 0.3750
class 0 clustering acc 0.625: correct 10/16
class 1 clustering acc 0.8125: correct 13/16
class 0: acc 0.5, correct 2/4
class 1: acc 0.5, correct 2/4
epoch:47


batch 19, loss: 1.5729, instance_loss: 0.3812, weighted_loss: 1.2154, label: 0, bag_size: 42
batch 39, loss: 1.8958, instance_loss: 0.5132, weighted_loss: 1.4810, label: 0, bag_size: 45


class 0 clustering acc 0.6979166666666666: correct 67/96
class 1 clustering acc 0.7708333333333334: correct 74/9

...

batch 19, loss: 0.0162, instance_loss: 0.2895, weighted_loss: 0.0982, label: 1, bag_size: 18
batch 39, loss: 0.2829, instance_loss: 0.1548, weighted_loss: 0.2444, label: 1, bag_size: 22


class 0 clustering acc 0.84375: correct 81/96
class 1 clustering acc 0.875: correct 84/96
Epoch: 352, train_loss: 0.2112, train_clustering_loss:  0.3708, train_error: 0.0625
class 0: acc 0.9090909090909091, correct 20/22
class 1: acc 0.9615384615384616, correct 25/26

Test Set, test_loss: 1.8514, test_error: 0.3750, auc: 0.4375
class 0 clustering acc 0.9375: correct 15/16
class 1 clustering acc 0.75: correct 12/16
class 0: acc 1.0, correct 4/4
class 1: acc 0.25, correct 1/4
epoch:353


batch 19, loss: 0.1701, instance_loss: 0.2731, weighted_loss: 0.2010, label: 1, bag_size: 18
batch 39, loss: 0.4490, instance_loss: 0.3687, weighted_loss: 0.4249, label: 0, bag_size: 14


class 0 clustering acc 0.9166666666666666: correct 88/96
class 1 clustering acc 0.9270833333333334: correct 89/96
Epoch: 353, train_l

epoch:365


batch 19, loss: 0.1883, instance_loss: 0.1830, weighted_loss: 0.1867, label: 0, bag_size: 12
batch 39, loss: 0.0097, instance_loss: 0.4247, weighted_loss: 0.1342, label: 1, bag_size: 34


class 0 clustering acc 0.8645833333333334: correct 83/96
class 1 clustering acc 0.8854166666666666: correct 85/96
Epoch: 365, train_loss: 0.1617, train_clustering_loss:  0.3388, train_error: 0.0208
class 0: acc 0.9545454545454546, correct 21/22
class 1: acc 1.0, correct 26/26

Test Set, test_loss: 1.8215, test_error: 0.3750, auc: 0.4375
class 0 clustering acc 0.9375: correct 15/16
class 1 clustering acc 0.75: correct 12/16
class 0: acc 1.0, correct 4/4
class 1: acc 0.25, correct 1/4
epoch:366


batch 19, loss: 0.5633, instance_loss: 0.2849, weighted_loss: 0.4798, label: 1, bag_size: 29
batch 39, loss: 0.0608, instance_loss: 0.5446, weighted_loss: 0.2059, label: 1, bag_size: 20


class 0 clustering acc 0.875: correct 84/96
class 1 clustering acc 0.8125: correct 78/96
Epoch: 366, train_loss:

batch 39, loss: 0.0287, instance_loss: 0.1891, weighted_loss: 0.0768, label: 1, bag_size: 23


class 0 clustering acc 0.90625: correct 87/96
class 1 clustering acc 0.8958333333333334: correct 86/96
Epoch: 377, train_loss: 0.1701, train_clustering_loss:  0.2947, train_error: 0.0208
class 0: acc 0.9545454545454546, correct 21/22
class 1: acc 1.0, correct 26/26

Test Set, test_loss: 1.9106, test_error: 0.3750, auc: 0.4375
class 0 clustering acc 0.9375: correct 15/16
class 1 clustering acc 0.75: correct 12/16
class 0: acc 1.0, correct 4/4
class 1: acc 0.25, correct 1/4
epoch:378


batch 19, loss: 0.1200, instance_loss: 0.1957, weighted_loss: 0.1427, label: 1, bag_size: 18
batch 39, loss: 0.6389, instance_loss: 0.4637, weighted_loss: 0.5863, label: 0, bag_size: 3


class 0 clustering acc 0.8958333333333334: correct 86/96
class 1 clustering acc 0.8125: correct 78/96
Epoch: 378, train_loss: 0.2579, train_clustering_loss:  0.3737, train_error: 0.0417
class 0: acc 0.9130434782608695, correct 21

batch 19, loss: 0.3680, instance_loss: 0.3314, weighted_loss: 0.3570, label: 1, bag_size: 22
batch 39, loss: 0.2689, instance_loss: 0.6371, weighted_loss: 0.3793, label: 1, bag_size: 20


class 0 clustering acc 0.9270833333333334: correct 89/96
class 1 clustering acc 0.8229166666666666: correct 79/96
Epoch: 390, train_loss: 0.1620, train_clustering_loss:  0.3477, train_error: 0.0208
class 0: acc 0.9565217391304348, correct 22/23
class 1: acc 1.0, correct 25/25

Test Set, test_loss: 1.8415, test_error: 0.3750, auc: 0.4375
class 0 clustering acc 0.9375: correct 15/16
class 1 clustering acc 0.75: correct 12/16
class 0: acc 1.0, correct 4/4
class 1: acc 0.25, correct 1/4
epoch:391


batch 19, loss: 0.0104, instance_loss: 0.4623, weighted_loss: 0.1460, label: 0, bag_size: 3
batch 39, loss: 0.0029, instance_loss: 0.2227, weighted_loss: 0.0689, label: 0, bag_size: 25


class 0 clustering acc 0.8645833333333334: correct 83/96
class 1 clustering acc 0.8541666666666666: correct 82/96
Epoch: 391,

/var/folders/45/fswzhp4j1qx7m6_9ty17lp500000gn/T/ipykernel_37562/3218699494.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append( { 'folds': '', 'stopping_epoch': np.mean(all_stopping_epoch),


In [15]:
# %% [markdown]
## Command Line Interfaces

# %% Creating a Parser
# import argparse
# parser = argparse.ArgumentParser()

# # %% Adding Arguments
# dir_project = '/SFS/user/ry/ohjihw/project/015'
# parser.add_argument( '--dir_project', type=str, default=dir_project )
# parser.add_argument( '--dir_code', type=str, default='code/python' )
# parser.add_argument( '--dir_oniput', type=str, default='oniput' )
# parser.add_argument( '--dir_split', type=str, default='split' )
# parser.add_argument( '--dir_result', type=str, default='result' )
# ###
# parser.add_argument( '--job_data', type=str, default='20230412-1-HM' )
# parser.add_argument( '--job_task', type=str, default='Glomeruli' )
# parser.add_argument( '--job_task', type=str, default='Tubules' )
# parser.add_argument( '--job_label', type=str, default='CASE' )
# ###
# parser.add_argument( '--cv_fold', type=int, default=1 )
# parser.add_argument( '--cv_val', type=float, default=0.0 )
# parser.add_argument( '--cv_test', type=float, default=0.0 )
# parser.add_argument( '--cv_train', type=float, default=1.0 )
# parser.add_argument( '--cv_equal', type=bool, default=True )
# ###
# parser.add_argument( '--opt', type=str, choices=[ 'adam', 'sgd' ], default='adam' )
# parser.add_argument( '--lr', type=float, default=1e-4, \
# 	help='learning rate (default: 1e-4)' )
# parser.add_argument( '--reg', type=float, default=1e-5, \
# 	help='weight decay (default: 1e-5)' )
# parser.add_argument( '--drop_out', action='store_true', default=True, \
# 	help='enabel dropout (p=0.25)' )
# parser.add_argument( '--weighted_sample', action='store_true', default=True, \
# 	help='enable weighted sampling' )
# parser.add_argument( '--max_epochs', type=int, default=400, \
# 	help='maximum number of epochs to train (default: 200)' )
# parser.add_argument( '--early_stopping', action='store_true', default=False, \
# 	help='enable early stopping' )
# ### CLAM
# parser.add_argument( '--model_type', type=str, choices=[ 'clam_sb', 'clam_mb', 'mil' ], default='clam_sb', \
# 	help='type of model (default: clam_sb, clam w/ single attention branch)' )
# parser.add_argument( '--bag_loss', type=str, choices=[ 'svm', 'ce' ], default='ce', \
# 	help='sample-level classification loss function (default: ce)' )
# parser.add_argument( '--inst_loss', type=str, choices=[ 'svm', 'ce', None ], default='svm', \
# 	help='instance-level clustering loss function (default: None)' )
# parser.add_argument( '--bag_weight', type=float, default=0.7, \
# 	help='clam: weight coefficient for bag-level loss (default: 0.7)' )
# parser.add_argument( '--B', type=int, default=10, \
# 	help='numbr of positive/negative patches to sample for clam' )
# parser.add_argument( '--model_size', type=str, choices=[ 'small', 'big' ], default='small', \
# 	help='size of model, does not affect mil' )
# parser.add_argument( '--no_inst_cluster', action='store_true', default=False, \
# 	help='disable instance-level clustering' )
# parser.add_argument( '--subtyping', action='store_true', default=False, \
# 	help='subtyping problem' )
# 	# In case you want to modify, maybe try first with 'bag_weight' and 'B'. - JO
# ### Miscellaneous
# parser.add_argument( '--log_data', action='store_true', default=True, \
# 	help='log data using tensorboard' )
# parser.add_argument( '--testing', action='store_true', default=False, \
# 	help='debugging tool' )

# %% Parsing Arguments
# in_ipython = hasattr(__builtins__,'__IPYTHON__')
# args = parser.parse_args("") if in_ipython else parser.parse_args()
# for ( key, value ) in vars(args).items():
# 	print( 'args' + '.' + key + ':', value )

# %%

# %% [markdown]
## Header

# %% Working Directory
# import os
# args.dir_code = os.path.join( args.dir_project, args.dir_code )
# print( 'args.dir_code:', args.dir_code )
# os.chdir( args.dir_code )
# args.dir_oniput = os.path.join( args.dir_project, args.dir_oniput )
# args.dir_oniput = os.path.join( args.dir_oniput, args.job_data )
# args.dir_oniput = os.path.join( args.dir_oniput, args.job_task )
# print( 'args.dir_ioput:', args.dir_oniput )
# args.dir_split = os.path.join( args.dir_oniput, args.dir_split )
# os.makedirs( args.dir_split, exist_ok=True )
# args.dir_result = os.path.join( args.dir_oniput, args.dir_result )
# os.makedirs( args.dir_result, exist_ok=True )

# %% General Modules
# import itertools
# import math
# import pdb
# import random
# ###
# import matplotlib.pylab as plt
# import numpy as np
# import pandas as pd
# import seaborn as sns

# # %% Special Modules - deep learning
# import torch
# import torch.nn.functional as F
# ###
# device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

# # %% Internal Modules
from models.model_clam import CLAM_MB, CLAM_SB
from utils.utils import *
from utils.file_utils import save_pkl, load_pkl

# %%

# %% [markdown]
## Preparation

# %%
def initiate_model(model_args, ckpt_path):
    print('Init Model')    
    model_dict = {
        "dropout": model_args['drop_out'].item(),
        'n_classes': model_args['n_classes'].item(), # model_args.n_classes,
        "k_sample": model_args['B'].item()
    }
    if model_args['model_size'].item() is not None and model_args['model_type'].item() in ['clam_sb', 'clam_mb']:
        model_dict.update({"size_arg": model_args['model_size'].item()})
    if model_args['model_type'].item() =='clam_sb':
        model = CLAM_SB(**model_dict)
    elif model_args['model_type'].item() =='clam_mb':
        model = CLAM_MB(**model_dict)
    else: # args.model_type == 'mil'
        if model_args['n_classes'].item() > 2:
            model = MIL_fc_mc(**model_dict)
        else:
            model = MIL_fc(**model_dict)
    print_network(model)
    ckpt = torch.load(ckpt_path)
    #print_network(ckpt)
    ckpt_clean = {}
    for key in ckpt.keys():
        if 'instance_loss_fn' in key:
            continue
        ckpt_clean.update({key.replace('.module', ''):ckpt[key]})
    model.load_state_dict(ckpt_clean, strict=True)
    model.relocate()
    model.eval()
    return model

# %%
def infer_single_slide(model, features, label, reverse_label_dict, k=1, instance_eval = False, label_tensor = None):
    features = features.to(device)
    with torch.no_grad():
        if isinstance(model, (CLAM_SB,)):
            #model_results_dict = model(features)
            logits, Y_prob, Y_hat, A, results_dict = model(features, instance_eval = instance_eval, label = label_tensor)
            print(results_dict)
            Y_hat = Y_hat.item()
            if isinstance(model, (CLAM_MB,)):
                A = A[Y_hat]
            A = A.view(-1, 1).cpu().numpy()
        else:
            raise NotImplementedError
        print('Y_hat: {}, Y: {}, Y_prob: {}'.format(reverse_label_dict[Y_hat], label, ["{:.4f}".format(p) for p in Y_prob.cpu().flatten()]))
        probs, ids = torch.topk(Y_prob, k)
        probs = probs[-1].cpu().numpy()
        ids = ids[-1].cpu().numpy()
        preds_str = np.array([reverse_label_dict[idx] for idx in ids])
    return ids, preds_str, probs, A, results_dict


# %%

# %% [markdown]
## Line by Line

# %%
def extract_attention_scores(args, cell_barcode_path, ckpt_path, feature_path, meta_path):
    ###
    cell_barcode_dict = load_pkl(cell_barcode_path)
    metadata = pd.read_csv(meta_path)
    files = os.listdir(feature_path)
    cell_weights = torch.Tensor()
    ###
    sample_list = []
    barcode_list = []
    results_dict_list = []
    ###
    for f in files:
        features = torch.load(os.path.join(feature_path, f))
        model = initiate_model(args, ckpt_path)
        sample_id = f.split('.')[0]
        if sample_id not in str( cell_barcode_dict.keys() ):
            continue
        cell_barcodes = cell_barcode_dict[ int( sample_id )]
        label = metadata[ metadata['slide_id'] == int( sample_id )].label.values[0]
        label_dict = {'CTR':0, 'CASE':1 }
        class_labels = list(label_dict.keys())
        class_encodings = list(label_dict.values())
        label_tensor = torch.tensor([label_dict[label]])
        reverse_label_dict = {class_encodings[i]: class_labels[i] for i in range(len(class_labels))} 
        ids, preds_str, probs, A, results_dict  = infer_single_slide(model, features, label, reverse_label_dict, k=1,
                                                        instance_eval = True, label_tensor= label_tensor)
        weights = F.softmax(torch.Tensor(A), dim = 0).view(1, -1)
        cell_weights = torch.cat((cell_weights, weights), dim = 1)
        sample_list.extend(np.repeat(sample_id, features.shape[0]))
        barcode_list.extend(cell_barcodes)
        results_dict_list.append(results_dict)
    return cell_weights, sample_list, barcode_list, results_dict_list

# %% [markdown]
## Main

# %%
# args.results_dir = '{}/{}_{}_{}'.format(
# 	args.dir_result,
# 	int( 100 * args.cv_val ), int( 100 * args.cv_test ), args.cv_fold
# )
###
# cell_barcode_path = "{}/insts_in_bags.pkl".format( args.dir_oniput )
# ckpt_path = "{}/s_0_checkpoint.pt".format( args.results_dir )
# feature_path = "{}/bags".format( args.dir_oniput )
# meta_path = "{}/bag.csv".format( args.dir_oniput )

cell_barcode_path = args['cell_barcode_path'].item()
ckpt_path = args['ckpt_path'].item()
feature_path = args['feature_path'].item()
meta_path = args['meta_path'].item()

#%%
# args.ckpt_path = ckpt_path # os.path.join(ckpt_path, f, 's_0_checkpoint.pt')
# args.k_sample = args.B #int(f.split('_B_')[1].split('_bag')[0])
cell_weights, sample_list, barcode_list, results_dict_list = extract_attention_scores( args,
                                                                        cell_barcode_path, #args.barcode_path,
                                                                        ckpt_path, #args.ckpt_path,
                                                                        feature_path, #args.feature_path, 
                                                                        meta_path) #args.meta_path 

# %%
DF = pd.DataFrame( {'bag': sample_list, 'inst': barcode_list, 'weight': cell_weights[0,:]} )
DF

# %%
path = "/Users/chitrakbanerjee/Desktop/omics/glom_pca/weights.csv"
DF.to_csv(path)

# %%

# %% [markdown]
## EoF


Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=50, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=8, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 1208
Total number of trainable parameters: 1208
{'instance_loss': tensor(0.2680), 'inst_labels': array([1, 1, 0, 

CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=50, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=8, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 1208
Total number of trainable parameters: 1208
{'instance_loss': tensor(0.7678), 'inst_labels': array([1, 1, 0, 0]), 'inst_

Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=50, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=8, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 1208
Total number of trainable parameters: 1208
{'instance_loss': tensor(0.0931), 'inst_labels': array([1, 1, 0, 

## Validation on unlabeled dataset manipulation and clean up

In [15]:
oob = pd.read_excel('validation_Glomeruli_combined.xlsx', sheet_name='in')

In [16]:
#oob.head()
oob1 = oob.rename({'Var1': 'bag', 'Var3': 'inst'}, axis='columns')

In [17]:
oob1.head()

,bag,Var2,inst,feat1,feat2,feat3,feat4,feat5,feat6,feat7,...,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315
0,1,1,V10_S_4_0.jpeg,0.79268,0.21862,0.084256,423850,2649.1,515.5,0.32402,...,129.910,192.750,244.990,281.920,312.910,338.62,361.15,381.45,400.68,421.15
1,1,2,V10_S_4_1.jpeg,0.80162,0.19187,0.086713,459760,2688.7,387.0,0.35926,...,115.590,175.060,227.430,265.220,298.050,325.95,348.84,368.02,387.03,406.84
2,1,3,V10_S_4_2.jpeg,0.81578,0.17478,0.074669,450370,2530.2,429.5,0.31225,...,124.220,180.420,231.190,266.180,295.550,323.42,347.42,368.17,387.23,407.42
3,1,4,V10_S_4_0.jpeg,0.79872,0.26091,0.002771,66788,16697.0,420.5,0.25032,...,43.322,62.301,76.722,88.781,99.322,108.84,117.75,126.32,135.32,154.32
4,1,5,V10_S_4_1.jpeg,0.75766,0.30022,0.003555,73692,10527.0,195.0,0.24646,...,46.740,65.744,80.744,94.577,106.500,117.42,127.70,137.29,148.16,168.74


In [18]:
oob1 = oob1.drop('Var2', axis = 1)

In [19]:
oob1['case'] = oob1['bag']

In [20]:
oob1.to_csv('oob_inst.csv', index = False)

## OOB validation

In [16]:
data_oob = pd.read_csv('oob_pca_inst.csv')

In [17]:
data_oob.head(3)

,bag,inst,img,case,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,...,principal component 41,principal component 42,principal component 43,principal component 44,principal component 45,principal component 46,principal component 47,principal component 48,principal component 49,principal component 50
0,1,1,V10_S_4_0.jpeg,1,15.734046,-10.160821,6.036202,4.078293,-1.660852,-2.019842,...,0.648651,0.732862,0.423890,-0.509064,1.444311,0.144973,0.458129,-1.709515,-0.608201,1.318741
1,1,2,V10_S_4_1.jpeg,1,16.039143,-12.042370,6.251794,4.663722,-0.697945,-1.065548,...,0.717671,0.628522,0.511682,-0.494698,0.668467,0.207231,0.894919,-1.653128,0.036238,1.294361
2,1,3,V10_S_4_2.jpeg,1,17.245286,-12.156861,8.130170,3.412733,-1.402968,-0.708747,...,0.450169,0.913126,0.788608,0.292066,1.020464,-0.336748,1.320325,-1.691266,0.641698,1.641376


In [16]:
data_oob.head(3)

,bag,inst,case,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,...,principal component 41,principal component 42,principal component 43,principal component 44,principal component 45,principal component 46,principal component 47,principal component 48,principal component 49,principal component 50
0,1,V10_S_4_0.jpeg,1,15.734046,-10.160821,6.036202,4.078293,-1.660852,-2.019841,0.515025,...,0.706494,-0.553590,0.568886,-0.601731,1.479422,0.110378,0.086540,-1.507616,-0.541741,0.540392
1,1,V10_S_4_1.jpeg,1,16.039143,-12.042370,6.251794,4.663722,-0.697945,-1.065548,2.583616,...,0.778605,-0.433178,0.604433,-0.567452,0.710257,0.039526,0.575932,-1.476300,0.143409,0.827610
2,1,V10_S_4_2.jpeg,1,17.245286,-12.156861,8.130170,3.412733,-1.402968,-0.708747,2.153190,...,0.509157,-0.645826,0.966872,0.212345,1.092971,-0.559418,0.794341,-1.584540,0.366599,1.485853


In [18]:
# %% Assign into NumPy Array
insts_oob = np.array( data_oob['inst'] )
bags_oob = np.array( data_oob['bag'] )
# cases = np.array( dataInst.iloc[:,2] )
# labels = np.array( dataInst.iloc[:,3] )
feats_oob = np.array( data_oob.loc[:, data_oob.columns.str.startswith('principal')] )

# %% PyTorch Tensor
# label2bool = np.array([True if l == 'CASE' else False for l in labels])
data_t_oob = TensorDataset(torch.Tensor(feats_oob)) #, torch.Tensor(label2bool))

In [19]:
# %% Bag Level Data
dataBag_oob = data_oob[['bag', 'case']]
print( dataBag_oob.shape )
###
dataBag_oob = dataBag_oob.drop_duplicates()
dataBag_oob = dataBag_oob.rename( columns={'bag': 'slide_id', 'case': 'case_id'} )
dataBag_oob.to_csv("bag_oob.csv", index = False)

(414, 2)


In [20]:
# %% OnI: Bag Level Data
# path = "{}/bag.csv".format( args.dir_oniput )
# print(path)
# dataBag.to_csv( path, index=False )

# %% List of Bags
bags_unique_oob = np.array(dataBag_oob['slide_id'])
print(bags_unique_oob)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


In [21]:
# %% Instances in Every Bag
dir = "{}/bags_oob".format('/Users/chitrakbanerjee/Desktop/omics/glom_pca')
print(dir)
os.makedirs( dir, exist_ok=True )
###
insts_in_bags_oob = {}
for b in bags_unique_oob: # this iterates 56 times
    # array of 1:56
    insts_sub = np.where(bags_oob == b)[0]
    # for each of 56 subjects tensor of features
    feats_sub = data_t_oob.tensors[0][insts_sub,:]
    # insts in bag
    insts_in_bags_oob[b] = insts_oob[insts_sub]
    path = "{}/{}.pt".format( dir, b )
    #print(path)
    torch.save( feats_sub, path )
    #print(insts_sub, feats_sub, insts_in_bags_oob[b])
    #print(insts_sub.shape, feats_sub.shape, insts_in_bags[b].shape)

###### this code is redundant    
# %% Cell Counts
# for ( key, values ) in insts_in_bags.items():
# 	print( str(key) + ':' + str( len(values) ) )

# # %%
path = "/Users/chitrakbanerjee/Desktop/omics/glom_pca/insts_in_bags_oob.pkl"
# print(path)
save_pkl( path, insts_in_bags_oob )

# # %%

# # %% [markdown]
# ## EoF

/Users/chitrakbanerjee/Desktop/omics/glom_pca/bags_oob


In [22]:
args = pd.DataFrame({'dir_project': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'dir_code': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'results_dir': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/result/',
        'dir_oniput': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/',
        'dir_split' : '/Users/chitrakbanerjee/Desktop/omics/glom_pca/split/',
        'dir_result' : '/Users/chitrakbanerjee/Desktop/omics/glom_pca/result/',
        'cell_barcode_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/insts_in_bags_oob.pkl',
        'ckpt_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/0_1/s_0_checkpoint.pt',
        'feature_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/bags_oob',
        'meta_path': '/Users/chitrakbanerjee/Desktop/omics/glom_pca/bag_oob.csv',
        'cv_fold': 5,
        'cv_val': 0.2,
        'cv_test': 0.2,
        'cv_train': 0.6,
        'cv_equal': True,
        'opt': 'adam',
        'log_data': True,
        'n_classes': 2,
        'lr': 1e-4,
        'reg': 1e-5,
        'drop_out': True,
        'weighted_sample': True,
        'max_epochs': 400,
        'early_stopping': False,
        'model_type': 'clam_mb',
        'bag_loss': 'svm',
        'inst_loss': 'ce',
        'bag_weight': 0.7,
        'B': 2,
        'model_size': 'tiny',
        'no_inst_cluster': False,
        'subtyping': False,
        'log_data': True,
        'testing': False
       }, index = [0])

In [23]:
from models.model_clam import CLAM_MB, CLAM_SB
from utils.utils import *
from utils.file_utils import save_pkl, load_pkl




def initiate_model(model_args, ckpt_path):
    print('Init Model')    
    model_dict = {
        "dropout": model_args['drop_out'].item(),
        'n_classes': model_args['n_classes'].item(), # model_args.n_classes,
        "k_sample": model_args['B'].item()
    }
    if model_args['model_size'].item() is not None and model_args['model_type'].item() in ['clam_sb', 'clam_mb']:
        model_dict.update({"size_arg": model_args['model_size'].item()})
    if model_args['model_type'].item() =='clam_sb':
        model = CLAM_SB(**model_dict)
    elif model_args['model_type'].item() =='clam_mb':
        model = CLAM_MB(**model_dict)
    else: # args.model_type == 'mil'
        if model_args['n_classes'].item() > 2:
            model = MIL_fc_mc(**model_dict)
        else:
            model = MIL_fc(**model_dict)
    print_network(model)
    ckpt = torch.load(ckpt_path)
    #print_network(ckpt)
    ckpt_clean = {}
    for key in ckpt.keys():
        if 'instance_loss_fn' in key:
            continue
        ckpt_clean.update({key.replace('.module', ''):ckpt[key]})
    model.load_state_dict(ckpt_clean, strict=True)
    model.relocate()
    model.eval()
    return model

# %%
def infer_single_slide(model, features, k=1, instance_eval = False, label_tensor = None):
    features = features.to(device)
    with torch.no_grad():
        if isinstance(model, (CLAM_SB,)):
            #model_results_dict = model(features)
            logits, Y_prob, Y_hat, A, results_dict = model(features, instance_eval = instance_eval)
            print(results_dict)
            Y_hat = Y_hat.item()
            if isinstance(model, (CLAM_MB,)):
                A = A[Y_hat]
            A = A.view(-1, 1).cpu().numpy()
        else:
            raise NotImplementedError
        print('Y_prob: {}', ["{:.4f}".format(p) for p in Y_prob.cpu().flatten()])
        probs, ids = torch.topk(Y_prob, k)
        probs = probs[-1].cpu().numpy()
        ids = ids[-1].cpu().numpy()
        #preds_str = np.array([reverse_label_dict[idx] for idx in ids])
    return ids, probs, A, results_dict


# %%

# %% [markdown]
## Line by Line

# %%
def extract_attention_scores(args, cell_barcode_path, ckpt_path, feature_path, meta_path):
    ###
    cell_barcode_dict = load_pkl(cell_barcode_path)
    metadata = pd.read_csv(meta_path)
    files = os.listdir(feature_path)
    cell_weights = torch.Tensor()
    ###
    sample_list = []
    barcode_list = []
    results_dict_list = []
    ###
    for f in files:
        features = torch.load(os.path.join(feature_path, f))
        model = initiate_model(args, ckpt_path)
        sample_id = f.split('.')[0]
        if sample_id not in str( cell_barcode_dict.keys() ):
            continue
        cell_barcodes = cell_barcode_dict[ int( sample_id )]
        #label = metadata[ metadata['slide_id'] == int( sample_id )].label.values[0]
        #label_dict = {'CTR':0, 'CASE':1 }
        #class_labels = list(label_dict.keys())
        #class_encodings = list(label_dict.values())
        #label_tensor = torch.tensor([label_dict[label]])
        #reverse_label_dict = {class_encodings[i]: class_labels[i] for i in range(len(class_labels))} 
        ids, probs, A, results_dict  = infer_single_slide(model, features, k=1,
                                                        instance_eval = False, label_tensor= None)
        weights = F.softmax(torch.Tensor(A), dim = 0).view(1, -1)
        cell_weights = torch.cat((cell_weights, weights), dim = 1)
        sample_list.extend(np.repeat(sample_id, features.shape[0]))
        barcode_list.extend(cell_barcodes)
        results_dict_list.append(results_dict)
    return cell_weights, sample_list, barcode_list, results_dict_list

# %% [markdown]
## Main

# %%
# args.results_dir = '{}/{}_{}_{}'.format(
# 	args.dir_result,
# 	int( 100 * args.cv_val ), int( 100 * args.cv_test ), args.cv_fold
# )
###
# cell_barcode_path = "{}/insts_in_bags.pkl".format( args.dir_oniput )
# ckpt_path = "{}/s_0_checkpoint.pt".format( args.results_dir )
# feature_path = "{}/bags".format( args.dir_oniput )
# meta_path = "{}/bag.csv".format( args.dir_oniput )

cell_barcode_path = args['cell_barcode_path'].item()
ckpt_path = args['ckpt_path'].item()
feature_path = args['feature_path'].item()
meta_path = args['meta_path'].item()

#%%
# args.ckpt_path = ckpt_path # os.path.join(ckpt_path, f, 's_0_checkpoint.pt')
# args.k_sample = args.B #int(f.split('_B_')[1].split('_bag')[0])
cell_weights, sample_list, barcode_list, results_dict_list = extract_attention_scores( args,
                                                                        cell_barcode_path, #args.barcode_path,
                                                                        ckpt_path, #args.ckpt_path,
                                                                        feature_path, #args.feature_path, 
                                                                        meta_path) #args.meta_path 

# %%
DF = pd.DataFrame( {'bag': sample_list, 'inst': barcode_list, 'weight': cell_weights[0,:]} )
DF

# %%
path = "/Users/chitrakbanerjee/Desktop/omics/glom_pca/weights_oob.csv"
DF.to_csv(path, index = False)

# %%

# %% [markdown]
## EoF


Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=50, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=8, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 1208
Total number of trainable parameters: 1208
{}
Y_prob: {} ['0.7059', '0.2941']
Init Model
CLAM_MB(
  (attenti

{}
Y_prob: {} ['0.4913', '0.5087']
Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=50, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=8, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=16, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 1208
Total number of trainable parameters: 1208
{}
Y_prob: {} ['0.9681', '0.03

In [24]:
unlabeled = pd.read_csv('weights_oob_glom.csv')

In [25]:
unlabeled.head()

,bag,inst,weight
0,19,1,0.036985
1,19,2,0.042359
2,19,3,0.083695
3,19,4,0.053932
4,19,5,0.095573


In [26]:
unlabeled.head()

,bag,inst,weight
0,19,1,0.036985
1,19,2,0.042359
2,19,3,0.083695
3,19,4,0.053932
4,19,5,0.095573


In [27]:
unlabeled.groupby('bag')['weight'].sum()

bag
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
Name: weight, dtype: float64

In [37]:
x = torch.load('/Users/chitrakbanerjee/Desktop/omics/new_dev/bags_oob/30.pt')

In [38]:
x.shape

torch.Size([22, 316])

In [39]:
x

tensor([[8.3765e-01, 1.8746e-01, 3.3689e-02,  ..., 3.6231e+02, 3.9483e+02,
         3.0000e+01],
        [7.9006e-01, 2.5811e-01, 6.0353e-02,  ..., 3.4161e+02, 3.7819e+02,
         3.0000e+01],
        [7.8716e-01, 2.4623e-01, 7.3277e-02,  ..., 3.9955e+02, 4.2055e+02,
         3.0000e+01],
        ...,
        [7.4368e-01, 3.1099e-01, 7.3491e-02,  ..., 2.5052e+02, 2.6366e+02,
         3.0000e+01],
        [7.4427e-01, 3.4818e-01, 3.6882e-02,  ..., 2.6038e+02, 2.8487e+02,
         3.0000e+01],
        [7.1670e-01, 3.5653e-01, 9.2368e-02,  ..., 3.1335e+02, 3.3705e+02,
         3.0000e+01]])

In [24]:
load_pkl('insts_in_bags.pkl')

{1: array(['10_S_3_0.jpeg', '10_S_3_1.jpeg', '10_S_3_2.jpeg', '10_S_3_3.jpeg',
        '10_S_3_0.jpeg', '10_S_3_1.jpeg', '10_S_3_2.jpeg'], dtype=object),
 2: array(['11_S_3_0.jpeg', '11_S_3_0.jpeg', '11_S_3_1.jpeg', '11_S_3_2.jpeg',
        '11_S_3_3.jpeg', '11_S_3_4.jpeg', '11_S_3_5.jpeg', '11_S_3_6.jpeg',
        '11_S_3_7.jpeg'], dtype=object),
 3: array(['13_S_170019134-3_0.jpeg', '13_S_170019134-3_1.jpeg',
        '13_S_170019134-3_10.jpeg', '13_S_170019134-3_11.jpeg',
        '13_S_170019134-3_12.jpeg', '13_S_170019134-3_13.jpeg',
        '13_S_170019134-3_14.jpeg', '13_S_170019134-3_15.jpeg',
        '13_S_170019134-3_16.jpeg', '13_S_170019134-3_17.jpeg',
        '13_S_170019134-3_18.jpeg', '13_S_170019134-3_19.jpeg',
        '13_S_170019134-3_2.jpeg', '13_S_170019134-3_20.jpeg',
        '13_S_170019134-3_21.jpeg', '13_S_170019134-3_22.jpeg',
        '13_S_170019134-3_23.jpeg', '13_S_170019134-3_3.jpeg',
        '13_S_170019134-3_4.jpeg', '13_S_170019134-3_5.jpeg',
        '13_S

In [1]:
from topk.svm import SmoothTop1SVM

In [31]:
output = torch.randn(10, 1)

In [32]:
output

tensor([[ 0.5221],
        [ 0.0993],
        [ 1.2200],
        [ 0.2630],
        [ 0.0533],
        [-0.0296],
        [ 0.1782],
        [-0.8404],
        [-0.7691],
        [ 1.0615]])

In [33]:
output.view(1, -1)

tensor([[ 0.5221,  0.0993,  1.2200,  0.2630,  0.0533, -0.0296,  0.1782, -0.8404,
         -0.7691,  1.0615]])

In [34]:
maxk = 1
torch.topk(output, 1)

torch.return_types.topk(
values=tensor([[ 0.5221],
        [ 0.0993],
        [ 1.2200],
        [ 0.2630],
        [ 0.0533],
        [-0.0296],
        [ 0.1782],
        [-0.8404],
        [-0.7691],
        [ 1.0615]]),
indices=tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]))

In [35]:
len(output.view(1, -1).shape)

2

In [23]:
maxk = 2
torch.topk(output, 1)[1][-1]

tensor([1])

In [16]:
args['cv_val'].item()

0.2

In [10]:
csv_path = os.path.join( '/Users/chitrakbanerjee/Desktop/omics/new_dev/split/', '{}_{}_{}_{}'.format(int( 100 * cv_val ), 
        int( 100 * cv_test ), cv_fold, f) + '.csv' )

In [11]:
d = dataset.return_splits(from_id=False, csv_path=csv_path, val_frac=0.2, test_frac=0.2 )

In [12]:
train_split = dataset.get_split_from_df(pd.read_csv(csv_path), 'train')

In [13]:
train_split[0]

(tensor([[7.6374e-01, 2.9191e-01, 4.6722e-02,  ..., 2.5534e+02, 2.6834e+02,
          2.8834e+02],
         [7.6065e-01, 3.1171e-01, 4.9396e-02,  ..., 4.6671e+02, 4.8443e+02,
          5.0843e+02],
         [7.1368e-01, 3.7529e-01, 6.6800e-02,  ..., 1.7643e+02, 1.8714e+02,
          2.0361e+02],
         ...,
         [7.5604e-01, 2.9954e-01, 4.7077e-02,  ..., 9.6748e+01, 1.0521e+02,
          1.1272e+02],
         [7.6887e-01, 3.0632e-01, 2.8374e-02,  ..., 1.6174e+02, 1.6955e+02,
          1.7655e+02],
         [7.6065e-01, 3.3915e-01, 2.0174e-02,  ..., 1.5948e+02, 1.6918e+02,
          1.7818e+02]]),
 0)

In [36]:
test_dataset[0][0]

tensor([[7.5540e-01, 2.9897e-01, 5.7978e-02,  ..., 4.0787e+02, 4.2369e+02,
         4.4192e+02],
        [7.3131e-01, 3.3108e-01, 7.8102e-02,  ..., 4.1476e+02, 4.3185e+02,
         4.5124e+02],
        [7.4528e-01, 3.3066e-01, 5.4547e-02,  ..., 4.1105e+02, 4.2760e+02,
         4.4498e+02],
        ...,
        [7.6000e-01, 3.0634e-01, 5.9379e-02,  ..., 3.5635e+02, 3.6736e+02,
         3.8192e+02],
        [7.7293e-01, 2.9386e-01, 3.7334e-02,  ..., 3.5137e+02, 3.6224e+02,
         3.7637e+02],
        [7.5152e-01, 3.4991e-01, 1.1812e-02,  ..., 1.3392e+02, 1.5302e+02,
         2.0238e+02]])